In [3]:
import gzip
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plac
from IPython.display import clear_output
import random
from statsmodels.distributions.empirical_distribution import ECDF
from scipy import stats as ss
import os
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
os.chdir('/gdrive/My Drive/IC_tales/code')
import disrupt as dr
import amutils as at

Mounted at /gdrive


<br><p>Carregando os dados</p><br>

In [0]:
dados = at.load_am_json_data()

In [0]:
G = at.build_graph(dados)

Atualizado


<br><p>Número de vértices e arestas de G:</p><br>

In [0]:
print("Vértices: ",G.number_of_nodes(),"\nArestas: ",G.number_of_edges())

Vértices:  32568 
Arestas:  119961


In [0]:
dtd_G = dr.compute_disruption(G)
clear_output()

In [0]:
dtd_G = dtd_G.dropna().copy()

In [0]:
ci_g = dr.get_confidence_disruption(dtd_G)

In [0]:
dtd_G['ci'] = 1-ci_g
dtd_G.head()

,ni_E,ni_D,ni,nj_E,nj_D,nj,nk_E,nk_D,nk,disruption,in,out,ci
mn0000748222,64.0,4.0,68.0,29.0,1.0,30.0,0.010425,98.0,14.0,0.0,0.0,0.0,0.9998
mn0000253142,53.0,2.0,55.0,33.0,2.0,35.0,0.014620,90.0,11.0,0.0,0.0,0.0,0.9727
mn0000754032,955.0,107.0,1062.0,377.0,53.0,430.0,0.225232,1492.0,27.0,0.0,0.0,0.0,1.0000
mn0000562304,170.0,4.0,174.0,189.0,2.0,191.0,-0.005152,365.0,21.0,0.0,0.0,0.0,0.1942
mn0000747445,42.0,1.0,43.0,48.0,1.0,49.0,-0.002022,92.0,12.0,0.0,0.0,0.0,0.2807


---

<br><h3>Disruption</h3>
<p>Calculando a disrupção de H: </p><br>

<br><p>Nesse bloco, able_edges recebe um set com todas as duplas de generos possíveis, exceto dupas com generos iguais.</p><br>

In [0]:
generos = at.build_reverse_index(dados)[1]
generos = set(generos.keys())
exceto = list()
for i in generos:
    exceto.append((i,i))
exceto = set(exceto)
able_edges = at.car(generos,generos) - exceto

<br> <p>O grafo G é formado por todas as arestas existentes.<br>
 O grafo H é só possui arestas entre nodes que são de gêneros diferentes.</p><br>

In [0]:
H = at.build_graph(dados, edges_to_consider = able_edges)

Atualizado


<br><p>Número de vértices e arestas de H:</p><br>

In [0]:
print("Vértices: ",H.number_of_nodes(),"\nArestas: ",H.number_of_edges())

Vértices:  13121 
Arestas:  27503


In [0]:
dtd_H = dr.compute_disruption(H)
clear_output()

In [0]:
dtd_H = dtd_H.dropna().copy()

In [0]:
ci_h = dr.get_confidence_disruption(dtd_H)

In [0]:
dtd_H['ci'] = 1-ci_h

In [0]:
dtd_G.head()

,ni_E,ni_D,ni,nj_E,nj_D,nj,nk_E,nk_D,nk,disruption,in,out,ci
mn0000748222,64.0,4.0,68.0,29.0,1.0,30.0,0.010425,98.0,14.0,0.0,0.0,0.0,0.9998
mn0000253142,53.0,2.0,55.0,33.0,2.0,35.0,0.014620,90.0,11.0,0.0,0.0,0.0,0.9727
mn0000754032,955.0,107.0,1062.0,377.0,53.0,430.0,0.225232,1492.0,27.0,0.0,0.0,0.0,1.0000
mn0000562304,170.0,4.0,174.0,189.0,2.0,191.0,-0.005152,365.0,21.0,0.0,0.0,0.0,0.1942
mn0000747445,42.0,1.0,43.0,48.0,1.0,49.0,-0.002022,92.0,12.0,0.0,0.0,0.0,0.2807


In [0]:
ecdf_g = ECDF(dtd_G['disruption'][dtd_G['ci'] < 0.05])
ecdf_h = ECDF(dtd_H['disruption'][dtd_H['ci'] < 0.05])

plt.plot(ecdf_g.x, 1-ecdf_g.y, label='g')
plt.plot(ecdf_h.x, 1-ecdf_h.y, label='h')
plt.legend()

ZeroDivisionError: float division by zero

In [0]:
ss.ks_2samp(dtd_G['disruption'][dtd_G['ci'] < 0.05],
            dtd_H['disruption'][dtd_H['ci'] < 0.05])

In [0]:
pass_both = (dtd_G.loc[dtd_H.index]['ci'] < 0.05) & (dtd_H['ci'] < 0.05)
pass_both = pass_both[pass_both == True].index

ss.spearmanr(dtd_G.loc[pass_both]['disruption'],
             dtd_H.loc[pass_both]['disruption'])


In [0]:
dtd_G.loc[dtd_H.index].shape

In [0]:
dtd_H.shape

In [0]:
print("Média em G:",round(dtd_G["disruption"].mean(),3)," Desvio padrão: ", round(dtd_G["disruption"].std(),3)," Mediana: ", round(dtd_G["disruption"].median(),3))
print("Média em H:",round(dtd_H["disruption"].mean(),3)," Desvio padrão: ", round(dtd_H["disruption"].std(),3)," Mediana: ", round(dtd_H["disruption"].median(),3))

---

<br><h3>PageRank</h3>
<p>Correlação com PageRank?</p><br>

<br><h4>Subgrafos de gêneros</h4>
<p>Calculamos a disruptividade para subgrafos onde todas as arestas possuem um determinado gênero na ponta ou na base.</p>
<p>Fazemos isso para cada gênero</p><br>

In [0]:
pagerank = pd.DataFrame(nx.pagerank(H).values(), index = nx.pagerank(H).keys())
dtd_H['pagerank']= pagerank
plt.scatter(dtd_H['disruption'][dtd_H['ci']<0.05],dtd_H['pagerank'][dtd_H['ci']<0.05],alpha=0.3)
ss.spearmanr(dtd_H['disruption'][dtd_H['ci']<0.05],dtd_H['pagerank'][dtd_H['ci']<0.05])

<br><p>E dentro de um gênero?</p><br>
<p>Aqui para cada gênero, eu pego o sub-grafo do grafo inteiro, onde nesse subgrafo todas as arestas possuem um artista desse gênero em uma de sua pontas ou nas duas</p>
<p>Após fazer isso eu calculo a diruptividade para cada um desses subgrafos e guardo isso tudo numa lista onde cada elemento é do tipo:<br> (genero,(subgrafo,tabela_de_disruptividade))</p>

In [0]:
data_genres = []

for genero in list(generos):
    print(genero)
    able_edges = list()

    for edge in set(at.car(generos,generos)):
        if edge[0]==genero or edge[1] == genero:
            able_edges.append(edge)

    able_edges = set(able_edges)
    
    print(len(list(able_edges)))
    
    grafo_genero = at.build_graph(dados, edges_to_consider = able_edges)
    
    print(grafo_genero.number_of_nodes())
    
    dtd_genero = dr.compute_disruption(grafo_genero)
    dtd_genero = dtd_genero.dropna()
    ci = dr.get_confidence_disruption(dtd_genero, prior=10)
    dtd_genero['ci'] = 1 - ci
    
    data_genres.append((genero,dict([('graph',grafo_genero),('disruption_table',dtd_genero)])))
    
clear_output()    
    

<br><p>O dicionário criado abaixo possui gêneros como chaves e em cada um de seus itens possui o grafo e a tabela de disruptividade daquele gênero</p><br>

In [0]:
data_genres = dict(data_genres)

<p>Exemplo:</p>

In [0]:
data_genres["Holiday"]

<br><p>ECDF da disrupção de cada um dos gêneros</p><br>

In [0]:
plt.figure(figsize=(10,8))
print('Estilos que não possuem nenhum artista com p-values baixo o suficiente:\n')
for genero in data_genres:
    try:
        ecdf_g = ECDF(data_genres[genero]['disruption_table']['disruption'][data_genres[genero]['disruption_table']['ci'] < 0.05])
        plt.plot(ecdf_g.x, ecdf_g.y, label=genero)
        plt.legend()
    except:
        print(genero)
plt.show()

<br><p>Calculando o pagerank do grafo de cada gênero e adicionando o resultado à uma nova coluna pagerank na tabela desse gênero</p><br>

<p>Repare que pagerank retorna uma coluna com mais linhas do que a nossa tabela.</p>
<p>Isso acontece porque o o algoritmo do pagerank também calcula o score dos nodes que nós tiramos por ter nans</p><br>

In [0]:
for genero in data_genres:
    data_genres[genero]['disruption_table']['pagerank'] = pd.DataFrame(nx.pagerank(data_genres[genero]['graph']).values(), index = nx.pagerank(data_genres[genero]['graph']).keys(),columns = ['pagerank'])
data_genres['Holiday']['disruption_table']

<br><p>E finalmente as correlções entre pagerank e disruptividade e os p-values de cada gênero do nosso grafo:</p><br>

In [0]:
correlation = []
for genero in data_genres:
    correlation.append(ss.spearmanr(data_genres[genero]['disruption_table']['pagerank'][data_genres[genero]['disruption_table']['ci']<0.05],
                                             data_genres[genero]['disruption_table']['disruption'][data_genres[genero]['disruption_table']['ci']<0.05]))
pd.DataFrame(np.array(correlation) , index = data_genres.keys(), columns=['correlation','p-value'])